In [1]:
import os
from pathlib import Path
import pandas as pd
import json
from epjson.epjson_idf import EpJsonIDF
from epjson.simulate_cluster import SimulateCluster
import datetime

def run_single_building(
        config_dict, 
        output_dir, 
        weight_map,
        resource_dir=Path("tests/resources"), 
        idf_dir=Path("tests/resources/idfs/"),
        eplus_loc=Path("C:\EnergyPlusV22-2-0"), 
        update_epsjon=True
        ):
    
    weights_df = pd.read_json(resource_dir / "shoebox-weights.json")
    weights_df = weights_df[weights_df["building_id"] == config_dict["building_id"]]
    idfs = set(weights_df.ShoeboxPath.to_list())

    ### MUTATIONS ###

    # Get retrofit scenario template data
    scenario_dir = resource_dir / f"templates_RETRO_{config_dict['retrofit_scenario']}_SCHED_{config_dict['schedules_scenario']}_LAB_{config_dict['lab_scenario']}.csv"
    features_df = pd.read_csv(scenario_dir)
    print(scenario_dir)
    with open(resource_dir / "template_features_map.json", "r") as f:
        features_map = json.load(f)
    with open(resource_dir / "schedule_bunches.json", "r") as f:
        schedules = json.load(f)
        
    if update_epsjon:
        for i, idf in enumerate(idfs):
            try:
                idf_path = idf_dir / idf
                epjson = EpJsonIDF.from_epjson(idf_path.with_suffix(".epjson"), eplus_loc=eplus_loc)
                features_series = features_df[(features_df.TYPOLOGY == config_dict["TYPOLOGY"]) & (features_df.AGE_ERA == config_dict["AGE_ERA"])].reset_index(drop=True)
                features_series = features_series.iloc[0]
                # Set DHW flow rate based on sb area
                peak_dhw_pp = features_series["DHWFlowRate"]
                occupants = features_series["PeopleDensity"] * (weights_df.iloc[i]["core_area"]+weights_df.iloc[i]["perim_area"])
                features_series["DHWFlowRate"] = peak_dhw_pp * occupants

                cols = set(features_series.index) & set(features_map.keys())

                epjson.replace_features(features_series[cols], features_map)
                epjson.update_schedules(schedules)
                epjson.add([(
                    "Output:SQLite",
                    {"Output:SQLite 1": {"option_type": "Simple"}}
                    )])
                epjson.epjson["ZoneInfiltration:DesignFlowRate"]["CoreInfiltration"].update(
                    {"air_changes_per_hour": 0}
                )
                epjson.epjson["WaterUse:Equipment"]["DHW Perimeter"].update(
                    {"peak_flow_rate": 0}
                )
                
                # new_path = replace_shoebox_path(idf_path).with_suffix(".epJSON")
                new_path = output_dir / Path(idf).with_suffix(".epJSON")
                epjson.save_as(new_path)
                epjson.save_idf(output_path=new_path.parent) #suffix="_test",

            except Exception as e:
                print(f"FAILED ON {idf}")
                raise e
    
    ### SIMULATION ###
    # Get new paths for updated shoeboxes
    # new_idfs = weights_df["ShoeboxPath"].apply(replace_shoebox_path)
    new_idfs = weights_df["ShoeboxPath"].apply(lambda x: output_dir / Path(x))
    weights_df["ShoeboxPath"] = new_idfs
    
    print("BUILDING CLUSTER")
    idf_cluster = SimulateCluster(
        idf_list = set(new_idfs),
        epw = config_dict["epw"],
        weights_df = weights_df,
        building_col = "building_id",
        eplus_location=eplus_loc,
        weight_map = weight_map
    )
    print("SIMULATING CLUSTER")
    errors = idf_cluster.parallel_simulate() # Fix errors dict

    print("PROCESSING CLUSTER RESULTS")
    ### POST-PROCESS ###
    r = idf_cluster.fetch_building_results_parallel([config_dict["building_id"]])

    return r, errors, idf_cluster

c:\Users\zoelh\AppData\Local\anaconda3\envs\ml-for-bem\lib\site-packages\outdated\utils.py:14: OutdatedPackageWarning: The package energy-pandas is out of date. Your version is 0.3.3, the latest is 0.3.4.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [2]:
config_dict = {
    "building_id": "62",
    "epw": "D:\\Users\\zoelh\\Dropbox (MIT)\\4.S42 Campus Decarb\\Energy Modeling\\epws\\BaselineFile.epw",
    "retrofit_scenario": "baseline",
    "schedules_scenario": "baseline",
    "lab_scenario": "baseline",
    "TYPOLOGY": "Residential",
    "AGE_ERA": 1945,
    "lab_weight": 0.268159426
}

sb_weight_map = {
    "Perimeter": "PerimeterAreaWeight",
    "Core": "CoreAreaWeight"
}

office_res, office_err, office_cluster = run_single_building(config_dict, 
                               weight_map = sb_weight_map,
                               output_dir = Path(os.getcwd()) / "eplus", 
                               idf_dir=Path("D:\\Users\\zoelh\\Dropbox (MIT)\\4.S42 Campus Decarb\\Energy Modeling\\idfs\\gis\\simple_shoeboxes"),
                               update_epsjon=True
                               )

C:\Users\zoelh\AppData\Local\Temp\ipykernel_589232\3925137982.py:48: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  epjson.replace_features(features_series[cols], features_map)
INFO:EPJSON:Setting PeopleDensity to 0.05
INFO:EPJSON:Setting EquipmentPowerDensity to 5.38
INFO:EPJSON:Setting LightsSchedule to RESltg
INFO:EPJSON:Setting VentilationPerPerson to 0.0
INFO:EPJSON:Setting DHWSupplyTemp to 52.0
INFO:EPJSON:Setting PartitionRValue to 0.227857143
INFO:EPJSON:Setting DHWFlowRate to 1.0560558140785207e-07
INFO:EPJSON:Setting InfiltrationType to AirChanges/Hour
INFO:EPJSON:Setting CoolingSetpoint to 23.89
INFO:EPJSON:Setting SensibleRecoveryEffectiveness to 0.7
INFO:EPJSON:Setting LatentRecoveryEffectiveness to 0.65
INFO:EPJSON:Setting DHWSchedule to APT_DHW_SCH
INFO:EPJSON:Setting CoolingAvailability to AllOn
INFO:EPJSON:Setting RecoverySettings to None
INFO:EPJSON:Setting WindowUValue to 5.78
INFO:EPJSON:Setting Roo

tests\resources\templates_RETRO_baseline_SCHED_baseline_LAB_baseline.csv
d:\Users\zoelh\GitRepos\shoebox-energy\eplus\62\north
62_north.epjson


INFO:EPJSON:EnergyPlus Starting
INFO:EPJSON:EnergyPlus, Version 22.2.0-c249759bad, YMD=2024.04.15 22:33
INFO:EPJSON:Converted input file format. Exiting.
INFO:EPJSON:EnergyPlus Run Time=00hr 00min  0.10sec
INFO:EPJSON:Command executed successfully.
C:\Users\zoelh\AppData\Local\Temp\ipykernel_589232\3925137982.py:48: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  epjson.replace_features(features_series[cols], features_map)
INFO:EPJSON:Setting PeopleDensity to 0.05
INFO:EPJSON:Setting EquipmentPowerDensity to 5.38
INFO:EPJSON:Setting LightsSchedule to RESltg
INFO:EPJSON:Setting VentilationPerPerson to 0.0
INFO:EPJSON:Setting DHWSupplyTemp to 52.0
INFO:EPJSON:Setting PartitionRValue to 0.227857143
INFO:EPJSON:Setting DHWFlowRate to 1.0560558140785207e-07
INFO:EPJSON:Setting InfiltrationType to AirChanges/Hour
INFO:EPJSON:Setting CoolingSetpoint to 23.89
INFO:EPJSON:Setting SensibleRecoveryEffectiveness to 0.7
INFO:EPJSON:

d:\Users\zoelh\GitRepos\shoebox-energy\eplus\62\south
62_south.epjson


INFO:EPJSON:EnergyPlus Starting
INFO:EPJSON:EnergyPlus, Version 22.2.0-c249759bad, YMD=2024.04.15 22:33
INFO:EPJSON:Converted input file format. Exiting.
INFO:EPJSON:EnergyPlus Run Time=00hr 00min  0.10sec
INFO:EPJSON:Command executed successfully.
C:\Users\zoelh\AppData\Local\Temp\ipykernel_589232\3925137982.py:48: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  epjson.replace_features(features_series[cols], features_map)
INFO:EPJSON:Setting PeopleDensity to 0.05
INFO:EPJSON:Setting EquipmentPowerDensity to 5.38
INFO:EPJSON:Setting LightsSchedule to RESltg
INFO:EPJSON:Setting VentilationPerPerson to 0.0
INFO:EPJSON:Setting DHWSupplyTemp to 52.0
INFO:EPJSON:Setting PartitionRValue to 0.227857143
INFO:EPJSON:Setting DHWFlowRate to 1.0560558140785207e-07
INFO:EPJSON:Setting InfiltrationType to AirChanges/Hour
INFO:EPJSON:Setting CoolingSetpoint to 23.89
INFO:EPJSON:Setting SensibleRecoveryEffectiveness to 0.7
INFO:EPJSON:

...

Fetching epjsons...: 100%|██████████| 4/4 [00:00<00:00, 30.83it/s]


SIMULATING CLUSTER


  0%|          | 0/4 [00:00<?, ?it/s]INFO:EPJSON:EnergyPlus Starting
INFO:EPJSON:EnergyPlus Starting
INFO:EPJSON:EnergyPlus, Version 22.2.0-c249759bad, YMD=2024.04.15 22:33
INFO:EPJSON:EnergyPlus, Version 22.2.0-c249759bad, YMD=2024.04.15 22:33
INFO:EPJSON:EnergyPlus Starting
INFO:EPJSON:EnergyPlus Starting
INFO:EPJSON:EnergyPlus, Version 22.2.0-c249759bad, YMD=2024.04.15 22:33
INFO:EPJSON:EnergyPlus, Version 22.2.0-c249759bad, YMD=2024.04.15 22:33
INFO:EPJSON:Adjusting Air System Sizing
INFO:EPJSON:Adjusting Standard 62.1 Ventilation Sizing
INFO:EPJSON:Initializing Simulation
INFO:EPJSON:Adjusting Air System Sizing
INFO:EPJSON:Adjusting Standard 62.1 Ventilation Sizing
INFO:EPJSON:Initializing Simulation
INFO:EPJSON:Adjusting Air System Sizing
INFO:EPJSON:Adjusting Standard 62.1 Ventilation Sizing
INFO:EPJSON:Initializing Simulation
INFO:EPJSON:Adjusting Air System Sizing
INFO:EPJSON:Adjusting Standard 62.1 Ventilation Sizing
INFO:EPJSON:Initializing Simulation
INFO:EPJSON:Reporting S

PROCESSING CLUSTER RESULTS


INFO:BATCH:There are 4 shoeboxes for 62
Fetching building results: 100%|██████████| 1/1 [00:01<00:00,  1.75s/it]


In [3]:
office_res[config_dict["building_id"]].sum()

C:\Users\zoelh\AppData\Local\Temp\ipykernel_589232\1685267888.py:1: FutureWarning: The default value of numeric_only in DataFrame.sum is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  office_res[config_dict["building_id"]].sum()


Zone Lights Electricity Energy                                                               106446.972622
Electric Equipment Electricity Energy                                                        217812.207671
Zone Ideal Loads Supply Air Total Heating Energy                                             469351.527947
Zone Ideal Loads Supply Air Total Cooling Energy                                             254229.948722
Water Use Equipment Heating Energy                                                            26756.293646
Zone Lights Electricity Energy_norm                                                              14.850618
Electric Equipment Electricity Energy_norm                                                       30.387392
Zone Ideal Loads Supply Air Total Heating Energy_norm                                            65.480117
Zone Ideal Loads Supply Air Total Cooling Energy_norm                                            35.468099
Water Use Equipment Heating Energy_no

Run all archetypes

In [ ]:
sbs = pd.read_json("tests/resources/shoebox-weights.json")
sbs = sbs.groupby(["TYPOLOGY", "AGE_ERA"]).first().reset_index()
res = {}
res_hourly = {}
for i, row in sbs.iterrows():
    config_dict = {
        "building_id": row.building_id,
        "epw": "D:\\Users\\zoelh\\Dropbox (MIT)\\4.S42 Campus Decarb\\Energy Modeling\\epws\\USA_MA_Boston-Logan.Intl.AP.725090_TMY3.epw",
        # "epw": "D:\\Users\\zoelh\\Dropbox (MIT)\\4.S42 Campus Decarb\\Energy Modeling\\epws\\BaselineFile.epw",
        "retrofit_scenario": "baseline",
        "schedules_scenario": "baseline",
        "lab_scenario": "baseline",
        "TYPOLOGY": row.TYPOLOGY,
        "AGE_ERA": row.AGE_ERA,
        "lab_weight": 0.8
    }
    office_res, office_err, office_cluster = run_single_building(config_dict, 
                                weight_map = sb_weight_map,
                                output_dir = Path(os.getcwd()) / "eplus", 
                                idf_dir=Path("D:\\Users\\zoelh\\Dropbox (MIT)\\4.S42 Campus Decarb\\Energy Modeling\\idfs\\gis\\simple_shoeboxes"),
                                update_epsjon=True
                                )
    print(row.TYPOLOGY, row.AGE_ERA)
    res[(row.TYPOLOGY, row.AGE_ERA, row.building_id)] = office_res[row.building_id].sum()
    office_res[row.building_id]["DateTime"] = pd.date_range(start=datetime.datetime(year=2025, day=1, month=1, hour=0), periods=8760, freq="H")
    office_res[row.building_id]["TYPOLOGY"] = row.TYPOLOGY
    office_res[row.building_id]["AGE_ERA"] = row.AGE_ERA
    res_hourly[(row.TYPOLOGY, row.AGE_ERA, row.building_id)] = office_res[row.building_id]
    

In [93]:
list(res_hourly.values())[0].columns

Index(['Zone Lights Electricity Energy',
       'Electric Equipment Electricity Energy',
       'Zone Ideal Loads Supply Air Total Heating Energy',
       'Zone Ideal Loads Supply Air Total Cooling Energy',
       'Water Use Equipment Heating Energy',
       'Zone Lights Electricity Energy_norm',
       'Electric Equipment Electricity Energy_norm',
       'Zone Ideal Loads Supply Air Total Heating Energy_norm',
       'Zone Ideal Loads Supply Air Total Cooling Energy_norm',
       'Water Use Equipment Heating Energy_norm', 'ModelArea', 'DateTime',
       'BuildingId'],
      dtype='object')

In [111]:
hourly_baseline = pd.DataFrame(columns=list(res_hourly.values())[0].columns)
for idx, df in res_hourly.items():
    df.set_index(["TYPOLOGY", "AGE_ERA", "BuildingId", "DateTime"])
    hourly_baseline = pd.concat([hourly_baseline, df], axis=0)

In [113]:
hourly_baseline.to_csv("baseline_results_samples.csv")

In [57]:
norm_cols = ['Zone Lights Electricity Energy_norm',
       'Electric Equipment Electricity Energy_norm',
       'Zone Ideal Loads Supply Air Total Heating Energy_norm',
       'Zone Ideal Loads Supply Air Total Cooling Energy_norm',
       'Water Use Equipment Heating Energy_norm']
pd.DataFrame.from_dict(res).T[norm_cols]

Zone Lights Electricity Energy_norm  \
Athletic             1945 51                            174.709437   
                     1980 W31                           152.870765   
Lab and Mixed Use    1945 1                             104.825664   
                     1980 13                            104.825662   
                     2015 12                            104.825661   
Office and Mixed Use 1945 50                            104.825659   
                     1980 W11                            50.665741   
Residential          1945 62                              15.11091   
                     1980 NW30                           15.110911   
                     2015 E37                             15.11091   
Support Dominant     1945 42                             26.570394   
                     1980 E23                            26.570394   
                     2015 42C                            26.570393   

                               Electric Equipment Electricity Energy_norm  \
Athletic             1945 51                                   436.773593   
                     1980 W31                                  436.773613   
Lab and Mixed Use    1945 1                                     480.45096   
                     1980 13                                   349.418875   
                     2015 12                                   105.553617   
Office and Mixed Use 1945 50                                   480.450939   
                     1980 W11                                  349.418901   
Residential          1945 62                                    30.920002   
                     1980 NW30                                  30.920005   
                     2015 E37                                   30.920003   
Support Dominant     1945 42                                   132.851972   
                     1980 E23                                  132.851969   
                     2015 42C                                  132.851964   

                               Zone Ideal Loads Supply Air Total Heating Energy_norm  \
Athletic             1945 51                                            11.736536      
                     1980 W31                                            0.671785      
Lab and Mixed Use    1945 1                                             15.917859      
                     1980 13                                             8.999106      
                     2015 12                                             13.82172      
Office and Mixed Use 1945 50                                            10.387457      
                     1980 W11                                           13.012336      
Residential          1945 62                                            88.650217      
                     1980 NW30                                          57.310861      
                     2015 E37                                           47.862422      
Support Dominant     1945 42                                            51.167153      
                     1980 E23                                           47.714292      
                     2015 42C                                           28.957659      

                               Zone Ideal Loads Supply Air Total Cooling Energy_norm  \
Athletic             1945 51                                            76.079774      
                     1980 W31                                           68.838449      
Lab and Mixed Use    1945 1                                            549.302325      
                     1980 13                                           444.702001      
                     2015 12                                           237.188842      
Office and Mixed Use 1945 50                                           558.040611      
                     1980 W11                                          370.145256      
Residential          1945 62      

In [114]:
sbs = pd.read_json("tests/resources/shoebox-weights.json")
sbs = sbs.groupby(["TYPOLOGY", "AGE_ERA"]).first().reset_index()
partialres = {}
for i, row in sbs.iterrows():
    config_dict = {
        "building_id": row.building_id,
        "epw": "D:\\Users\\zoelh\\Dropbox (MIT)\\4.S42 Campus Decarb\\Energy Modeling\\epws\\USA_MA_Boston-Logan.Intl.AP.725090_TMY3.epw",
        "retrofit_scenario": "partial",
        "schedules_scenario": "baseline",
        "lab_scenario": "baseline",
        "TYPOLOGY": row.TYPOLOGY,
        "AGE_ERA": row.AGE_ERA,
        "lab_weight": 0.8
    }
    office_res, office_err, office_cluster = run_single_building(config_dict, 
                                weight_map = sb_weight_map,
                                output_dir = Path(os.getcwd()) / "eplus", 
                                idf_dir=Path("D:\\Users\\zoelh\\Dropbox (MIT)\\4.S42 Campus Decarb\\Energy Modeling\\idfs\\gis\\simple_shoeboxes"),
                                update_epsjon=True
                                )
    print(row.TYPOLOGY, row.AGE_ERA)
    partialres[(row.TYPOLOGY, row.AGE_ERA, row.building_id)] = office_res[row.building_id].sum()
    

C:\Users\zoelh\AppData\Local\Temp\ipykernel_575388\124095025.py:48: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  epjson.replace_features(features_series[cols], features_map)
INFO:EPJSON:Setting VentilationPerArea to 5e-05
INFO:EPJSON:Setting DHWSchedule to DHWOffice
INFO:EPJSON:Setting OASchedule to AllOn
INFO:EPJSON:Setting DHWSupplyTemp to 60.0
INFO:EPJSON:Setting VentilationMode to None
INFO:EPJSON:Setting HeatingAvailability to AllOn
INFO:EPJSON:Setting WindowUValue to 3.5
INFO:EPJSON:Setting InfiltrationType to AirChanges/Hour
INFO:EPJSON:Setting PartitionRValue to 0.227857143
INFO:EPJSON:Setting CoolingAvailability to AllOn
INFO:EPJSON:Setting HeatingSchedule to HeatingSPSchedule
INFO:EPJSON:Setting LightsSchedule to AllOn
INFO:EPJSON:Setting Infiltration to 2.0
INFO:EPJSON:Setting SensibleRecoveryEffectiveness to 0.7
INFO:EPJSON:Setting CoolingSchedule to CoolingSPSchedule
INFO:EPJSON:Setting RoofRValue to 3.1

tests\resources\templates_RETRO_partial_SCHED_baseline_LAB_baseline.csv
d:\Users\zoelh\GitRepos\shoebox-energy\eplus\51\south
51_south.epjson


INFO:EPJSON:EnergyPlus Starting
INFO:EPJSON:EnergyPlus, Version 22.2.0-c249759bad, YMD=2024.04.15 10:04
INFO:EPJSON:Converted input file format. Exiting.
INFO:EPJSON:EnergyPlus Run Time=00hr 00min  0.11sec
INFO:EPJSON:Command executed successfully.
C:\Users\zoelh\AppData\Local\Temp\ipykernel_575388\124095025.py:48: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  epjson.replace_features(features_series[cols], features_map)
INFO:EPJSON:Setting VentilationPerArea to 5e-05
INFO:EPJSON:Setting DHWSchedule to DHWOffice
INFO:EPJSON:Setting OASchedule to AllOn
INFO:EPJSON:Setting DHWSupplyTemp to 60.0
INFO:EPJSON:Setting VentilationMode to None
INFO:EPJSON:Setting HeatingAvailability to AllOn
INFO:EPJSON:Setting WindowUValue to 3.5
INFO:EPJSON:Setting InfiltrationType to AirChanges/Hour
INFO:EPJSON:Setting PartitionRValue to 0.227857143
INFO:EPJSON:Setting CoolingAvailability to AllOn
INFO:EPJSON:Setting HeatingSchedule to Heat

d:\Users\zoelh\GitRepos\shoebox-energy\eplus\51\east
51_east.epjson


INFO:EPJSON:EnergyPlus Starting
INFO:EPJSON:EnergyPlus, Version 22.2.0-c249759bad, YMD=2024.04.15 10:04
INFO:EPJSON:Converted input file format. Exiting.
INFO:EPJSON:EnergyPlus Run Time=00hr 00min  0.10sec
INFO:EPJSON:Command executed successfully.
C:\Users\zoelh\AppData\Local\Temp\ipykernel_575388\124095025.py:48: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  epjson.replace_features(features_series[cols], features_map)
INFO:EPJSON:Setting VentilationPerArea to 5e-05
INFO:EPJSON:Setting DHWSchedule to DHWOffice
INFO:EPJSON:Setting OASchedule to AllOn
INFO:EPJSON:Setting DHWSupplyTemp to 60.0
INFO:EPJSON:Setting VentilationMode to None
INFO:EPJSON:Setting HeatingAvailability to AllOn
INFO:EPJSON:Setting WindowUValue to 3.5
INFO:EPJSON:Setting InfiltrationType to AirChanges/Hour
INFO:EPJSON:Setting PartitionRValue to 0.227857143
INFO:EPJSON:Setting CoolingAvailability to AllOn
INFO:EPJSON:Setting HeatingSchedule to Heat

...

  0%|          | 0/4 [00:00<?, ?it/s]INFO:EPJSON:EnergyPlus Starting
INFO:EPJSON:EnergyPlus Starting
INFO:EPJSON:EnergyPlus, Version 22.2.0-c249759bad, YMD=2024.04.15 10:06
INFO:EPJSON:EnergyPlus Starting
INFO:EPJSON:EnergyPlus Starting
INFO:EPJSON:EnergyPlus, Version 22.2.0-c249759bad, YMD=2024.04.15 10:06
INFO:EPJSON:EnergyPlus, Version 22.2.0-c249759bad, YMD=2024.04.15 10:06
INFO:EPJSON:EnergyPlus, Version 22.2.0-c249759bad, YMD=2024.04.15 10:06
INFO:EPJSON:Adjusting Air System Sizing
INFO:EPJSON:Adjusting Standard 62.1 Ventilation Sizing
INFO:EPJSON:Initializing Simulation
INFO:EPJSON:Adjusting Air System Sizing
INFO:EPJSON:Adjusting Standard 62.1 Ventilation Sizing
INFO:EPJSON:Initializing Simulation
INFO:EPJSON:Adjusting Air System Sizing
INFO:EPJSON:Adjusting Standard 62.1 Ventilation Sizing
INFO:EPJSON:Initializing Simulation
INFO:EPJSON:Adjusting Air System Sizing
INFO:EPJSON:Adjusting Standard 62.1 Ventilation Sizing
INFO:EPJSON:Initializing Simulation
INFO:EPJSON:Reporting S

PROCESSING CLUSTER RESULTS


INFO:BATCH:There are 4 shoeboxes for 42C
Fetching building results: 100%|██████████| 1/1 [00:01<00:00,  1.71s/it]

Support Dominant 2015



C:\Users\zoelh\AppData\Local\Temp\ipykernel_575388\646621354.py:22: FutureWarning: The default value of numeric_only in DataFrame.sum is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  partialres[(row.TYPOLOGY, row.AGE_ERA, row.building_id)] = office_res[row.building_id].sum()


In [60]:
pd.DataFrame.from_dict(partialres).T[norm_cols]

Zone Lights Electricity Energy_norm  \
Athletic             1945 51                             87.354719   
                     1980 W31                             78.61925   
Lab and Mixed Use    1945 1                              52.412832   
                     1980 13                             52.412831   
                     2015 12                              52.41283   
Office and Mixed Use 1945 50                              52.41283   
                     1980 W11                             25.33287   
Residential          1945 62                              7.555455   
                     1980 NW30                            7.555456   
                     2015 E37                             7.555455   
Support Dominant     1945 42                             13.285197   
                     1980 E23                            13.285197   
                     2015 42C                            13.285196   

                               Electric Equipment Electricity Energy_norm  \
Athletic             1945 51                                   436.773593   
                     1980 W31                                  436.773613   
Lab and Mixed Use    1945 1                                     480.45096   
                     1980 13                                   349.418875   
                     2015 12                                   105.553617   
Office and Mixed Use 1945 50                                   480.450939   
                     1980 W11                                  349.418901   
Residential          1945 62                                    30.920002   
                     1980 NW30                                  30.920005   
                     2015 E37                                   30.920003   
Support Dominant     1945 42                                   132.851972   
                     1980 E23                                  132.851969   
                     2015 42C                                  132.851964   

                               Zone Ideal Loads Supply Air Total Heating Energy_norm  \
Athletic             1945 51                                            14.052881      
                     1980 W31                                            0.034153      
Lab and Mixed Use    1945 1                                              11.00495      
                     1980 13                                             4.965824      
                     2015 12                                            11.053994      
Office and Mixed Use 1945 50                                             7.229004      
                     1980 W11                                            4.757419      
Residential          1945 62                                            82.104237      
                     1980 NW30                                           49.01215      
                     2015 E37                                           41.198976      
Support Dominant     1945 42                                            18.739557      
                     1980 E23                                           21.914194      
                     2015 42C                                           10.719523      

                               Zone Ideal Loads Supply Air Total Cooling Energy_norm  \
Athletic             1945 51                                             65.84975      
                     1980 W31                                           60.661892      
Lab and Mixed Use    1945 1                                             91.141196      
                     1980 13                                            63.203425      
                     2015 12                                            43.308154      
Office and Mixed Use 1945 50                                            59.578521      
                     1980 W11                                           69.199485      
Residential          1945 62      

In [ ]:
sbs = pd.read_json("tests/resources/shoebox-weights.json")
sbs = sbs.groupby(["TYPOLOGY", "AGE_ERA"]).first().reset_index()
fullres = {}
for i, row in sbs.iterrows():
    config_dict = {
        "building_id": row.building_id,
        "epw": "D:\\Users\\zoelh\\Dropbox (MIT)\\4.S42 Campus Decarb\\Energy Modeling\\epws\\USA_MA_Boston-Logan.Intl.AP.725090_TMY3.epw",
        "retrofit_scenario": "full",
        "schedules_scenario": "baseline",
        "lab_scenario": "baseline",
        "TYPOLOGY": row.TYPOLOGY,
        "AGE_ERA": row.AGE_ERA,
        "lab_weight": 0.8
    }
    office_res, office_err, office_cluster = run_single_building(config_dict, 
                                weight_map = sb_weight_map,
                                output_dir = Path(os.getcwd()) / "eplus", 
                                idf_dir=Path("D:\\Users\\zoelh\\Dropbox (MIT)\\4.S42 Campus Decarb\\Energy Modeling\\idfs\\gis\\simple_shoeboxes"),
                                update_epsjon=True
                                )
    print(row.TYPOLOGY, row.AGE_ERA)
    fullres[(row.TYPOLOGY, row.AGE_ERA, row.building_id)] = office_res[row.building_id].sum()
    

In [62]:
pd.DataFrame.from_dict(fullres).T[norm_cols]

Zone Lights Electricity Energy_norm  \
Athletic             1945 51                             87.354719   
                     1980 W31                             78.61925   
Lab and Mixed Use    1945 1                              52.412832   
                     1980 13                             52.412831   
                     2015 12                              52.41283   
Office and Mixed Use 1945 50                              52.41283   
                     1980 W11                             25.33287   
Residential          1945 62                              7.555455   
                     1980 NW30                            7.555456   
                     2015 E37                             7.555455   
Support Dominant     1945 42                             13.285197   
                     1980 E23                            13.285197   
                     2015 42C                            13.285196   

                               Electric Equipment Electricity Energy_norm  \
Athletic             1945 51                                   436.773593   
                     1980 W31                                  436.773613   
Lab and Mixed Use    1945 1                                     480.45096   
                     1980 13                                   349.418875   
                     2015 12                                   105.553617   
Office and Mixed Use 1945 50                                   480.450939   
                     1980 W11                                  349.418901   
Residential          1945 62                                    30.920002   
                     1980 NW30                                  30.920005   
                     2015 E37                                   30.920003   
Support Dominant     1945 42                                   132.851972   
                     1980 E23                                  132.851969   
                     2015 42C                                  132.851964   

                               Zone Ideal Loads Supply Air Total Heating Energy_norm  \
Athletic             1945 51                                                  0.0      
                     1980 W31                                                 0.0      
Lab and Mixed Use    1945 1                                                   0.0      
                     1980 13                                             0.002462      
                     2015 12                                             1.587394      
Office and Mixed Use 1945 50                                                  0.0      
                     1980 W11                                            0.040111      
Residential          1945 62                                            30.460753      
                     1980 NW30                                           12.12782      
                     2015 E37                                           11.881545      
Support Dominant     1945 42                                             6.900067      
                     1980 E23                                            6.243716      
                     2015 42C                                             4.43932      

                               Zone Ideal Loads Supply Air Total Cooling Energy_norm  \
Athletic             1945 51                                            61.736831      
                     1980 W31                                           59.307497      
Lab and Mixed Use    1945 1                                              88.42994      
                     1980 13                                            61.598259      
                     2015 12                                            41.221423      
Office and Mixed Use 1945 50                                            58.401299      
                     1980 W11                                           66.612453      
Residential          1945 62      

In [ ]:
sbs = pd.read_json("tests/resources/shoebox-weights.json")
sbs = sbs.groupby(["TYPOLOGY", "AGE_ERA"]).first().reset_index()
allfullres = {}
for i, row in sbs.iterrows():
    config_dict = {
        "building_id": row.building_id,
        "epw": "D:\\Users\\zoelh\\Dropbox (MIT)\\4.S42 Campus Decarb\\Energy Modeling\\epws\\USA_MA_Boston-Logan.Intl.AP.725090_TMY3.epw",
        "retrofit_scenario": "full",
        "schedules_scenario": "full",
        "lab_scenario": "baseline",
        "TYPOLOGY": row.TYPOLOGY,
        "AGE_ERA": row.AGE_ERA,
        "lab_weight": 0.8
    }
    office_res, office_err, office_cluster = run_single_building(config_dict, 
                                weight_map = sb_weight_map,
                                output_dir = Path(os.getcwd()) / "eplus", 
                                idf_dir=Path("D:\\Users\\zoelh\\Dropbox (MIT)\\4.S42 Campus Decarb\\Energy Modeling\\idfs\\gis\\simple_shoeboxes"),
                                update_epsjon=True
                                )
    print(row.TYPOLOGY, row.AGE_ERA)
    allfullres[(row.TYPOLOGY, row.AGE_ERA, row.building_id)] = office_res[row.building_id].sum()


In [64]:
pd.DataFrame.from_dict(allfullres).T[norm_cols]

Zone Lights Electricity Energy_norm  \
Athletic             1945 51                             87.354719   
                     1980 W31                             78.61925   
Lab and Mixed Use    1945 1                              32.648827   
                     1980 13                             32.648826   
                     2015 12                             32.648826   
Office and Mixed Use 1945 50                             32.648825   
                     1980 W11                            32.648829   
Residential          1945 62                              7.555455   
                     1980 NW30                            7.555456   
                     2015 E37                             7.555455   
Support Dominant     1945 42                             27.207356   
                     1980 E23                            27.207355   
                     2015 42C                            27.207354   

                               Electric Equipment Electricity Energy_norm  \
Athletic             1945 51                                   436.773593   
                     1980 W31                                  436.773613   
Lab and Mixed Use    1945 1                                     480.45096   
                     1980 13                                   349.418875   
                     2015 12                                   105.553617   
Office and Mixed Use 1945 50                                   480.450939   
                     1980 W11                                  349.418901   
Residential          1945 62                                    30.920002   
                     1980 NW30                                  30.920005   
                     2015 E37                                   30.920003   
Support Dominant     1945 42                                   132.851972   
                     1980 E23                                  132.851969   
                     2015 42C                                  132.851964   

                               Zone Ideal Loads Supply Air Total Heating Energy_norm  \
Athletic             1945 51                                                  0.0      
                     1980 W31                                                 0.0      
Lab and Mixed Use    1945 1                                                   0.0      
                     1980 13                                                  0.0      
                     2015 12                                             0.100986      
Office and Mixed Use 1945 50                                                  0.0      
                     1980 W11                                                 0.0      
Residential          1945 62                                            17.905335      
                     1980 NW30                                           5.145644      
                     2015 E37                                            4.954598      
Support Dominant     1945 42                                             0.386677      
                     1980 E23                                            0.336192      
                     2015 42C                                            0.211382      

                               Zone Ideal Loads Supply Air Total Cooling Energy_norm  \
Athletic             1945 51                                            61.677695      
                     1980 W31                                           59.276548      
Lab and Mixed Use    1945 1                                             71.013608      
                     1980 13                                            47.978383      
                     2015 12                                            33.496247      
Office and Mixed Use 1945 50                                            44.876122      
                     1980 W11                                           55.365314      
Residential          1945 62      